In [1]:
import pandas as pd

from pandas_datareader import data as pdr

import yfinance as yf
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import scipy
import scipy.stats as st

# Create our portfolio of equities
tickers = ['^GSPC']
 
# Set the investment weightings (I arbitrarily picked for example)
weights = np.array([1])
 
# Set an initial investment level
initial_investment = 100000
 
# Download closing prices
data = pdr.get_data_yahoo(tickers, start="2010-01-01", end=datetime.date.today())['Adj Close']
 
#From the closing prices, calculate periodic returns

#Simple Return

#returns = data.pct_change().dropna()

returns = np.log(data/data.shift(1)).dropna()

returns

Symbols,^GSPC
Date,
2010-01-05,0.003111
2010-01-06,0.000545
2010-01-07,0.003993
2010-01-08,0.002878
2010-01-11,0.001745
...,...
2022-05-31,-0.006294
2022-06-01,-0.007511
2022-06-02,0.018263


In [2]:
r = returns.mean().dot(weights)

vol = np.sqrt(weights.T.dot(returns.cov()).dot(weights))

print('mean return: {: 4f}, volatility: {: 4f}'.format(r, vol))

mean return:  0.000413, volatility:  0.011035


In [3]:

#daily_returns = pd.Series((1e5*returns)@weights,index=returns.index)

daily_returns = pd.DataFrame(returns.dot(weights) * initial_investment)

#daily_returns.sort_values(inplace = True)


daily_returns['Quarter'] = pd.PeriodIndex(daily_returns.index, freq='Q')

daily_returns.rename(columns={ daily_returns.columns[0]: "Returns" }, inplace = True)


daily_returns = daily_returns[['Quarter', 'Returns']]

daily_returns

,Quarter,Returns
Date,,
2010-01-05,2010Q1,311.083197
2010-01-06,2010Q1,54.537189
2010-01-07,2010Q1,399.321835
2010-01-08,2010Q1,287.758308
2010-01-11,2010Q1,174.523163
...,...,...
2022-05-31,2022Q2,-629.413519
2022-06-01,2022Q2,-751.090504
2022-06-02,2022Q2,1826.322542


In [6]:
        
dct = {}
for idx, v in enumerate(daily_returns['Quarter'].unique()):
    dct[f'{v}'] = daily_returns.loc[daily_returns['Quarter'] == v]
    
dct['2010Q1'].head(5)

,Quarter,Returns
Date,,
2010-01-05,2010Q1,311.083197
2010-01-06,2010Q1,54.537189
2010-01-07,2010Q1,399.321835
2010-01-08,2010Q1,287.758308
2010-01-11,2010Q1,174.523163


In [45]:
volatility = {}

for i in list(daily_returns['Quarter'].unique().strftime('%YQ%q')):
    volatility[f'{i}'] = (dct[i]['Returns'].std())
    

current_vol = volatility[sorted(volatility.keys())[-1]]

norm_vol = current_vol/(list(volatility.values()))

norm_vol = pd.DataFrame(norm_vol, index = daily_returns['Quarter'].unique(), columns = ['Normalisation'])

norm_vol.head(5)

,Normalisation
2010Q1,1.937477
2010Q2,1.107851
2010Q3,1.526718
2010Q4,2.341592
2011Q1,2.086082


In [52]:
daily_returns = daily_returns.merge(norm_vol, left_on='Quarter', right_index=True, how='left')



In [55]:
daily_returns['Normalised Return'] = daily_returns['Returns'] * daily_returns['Normalisation']

daily_returns.head(5)

,Quarter,Returns,Normalisation,Normalised Return
Date,,,,
2010-01-05,2010Q1,311.083197,1.937477,602.716622
2010-01-06,2010Q1,54.537189,1.937477,105.664563
2010-01-07,2010Q1,399.321835,1.937477,773.676978
2010-01-08,2010Q1,287.758308,1.937477,557.525179
2010-01-11,2010Q1,174.523163,1.937477,338.134660


In [70]:
norm_return = np.array(daily_returns['Normalised Return'])
#list(norm_return)

In [73]:
def ES(c,x):
    Var=np.percentile(c,(1-x)*100)
    ES=c[c<=Var].mean()
    
    return [Var, abs(ES)]

ES(norm_return, 0.95)

ES_daily = ES(norm_return, 0.95)[1]

ES_daily


3985.524338666538

In [74]:
num_days = 10

ES_nday = []

for i in range(1, num_days+1):
    ES_nday.append(np.round(ES_daily * np.sqrt(i),2))
    print(str(i) + ' day ES at 95% confidence level: ' + str(np.round(ES_daily * np.sqrt(i),2)))


1 day ES at 95% confidence level: 3985.52
2 day ES at 95% confidence level: 5636.38
3 day ES at 95% confidence level: 6903.13
4 day ES at 95% confidence level: 7971.05
5 day ES at 95% confidence level: 8911.9
6 day ES at 95% confidence level: 9762.5
7 day ES at 95% confidence level: 10544.71
8 day ES at 95% confidence level: 11272.77
9 day ES at 95% confidence level: 11956.57
10 day ES at 95% confidence level: 12603.33
